# Group Assignment AY 2025-2026
## Getting you started (Part 1)

Welcome to the first notebook in this three-part series designed to help you get started on your group assignment. In this series, we'll guide you through various stages of data exploration, cleaning, and analysis to help you gain insights into the dataset and meet the requirements of your assignment.

### What to Expect in this Notebook
In this first notebook, we will:

- Understand the dataset – Examine the columns and data types, and get familiar with the dataset's structure.
- Basic data cleaning – Handle missing values, remove duplicates, and prepare the dataset for analysis.
- Initial exploration – Perform basic descriptive statistics and visualizations to uncover key patterns in the data.

By the end of this notebook, you and your group will have a solid foundation to get started on the group assignment and prepare for more advanced analysis in the following notebooks.

In [ ]:
# Import packages
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [ ]:
# Define directory to look at
data_dir = "./Data/"

# Let's have a look
os.listdir(data_dir)

In [ ]:
# NOTE: 'Trips' is not a file, but a folder!
# NOTE: if you are working on a mac, you may have a .DS_Store file in your directory
# this is a hidden file that is created by macOS

# We can select a certain file type (here: .csv) by running the following command:
dir_list = [obs for obs in os.listdir(data_dir) if ".csv" in obs]
dir_list

In [ ]:
# Since 'Trips' is not a file, but a folder we access its contents by doing the following:
folder_dir = './Data/Trips/'
folder_list = [obs for obs in os.listdir(folder_dir) if ".DS" not in obs]
folder_list

## Overview

<img src="./Data/Overview_NMBS_data.png" align="center" width="800"/>

\* Only travelers with a subscription were asked to participate in the satisfaction survey. You can also look at external data sources w.r.t. satisfaction of the stations themselves.

\** It is possible that a station is mentioned but that the train didn’t stop there, it just went through the station (e.g., IC lines)

## 1. Tickets & Subscriptions
These are two of the main files for the group assignment. As you can see in the ERD above, they are at the heart of the overview.

In [ ]:
# Import tickets and subscriptions datasets
tickets_df = pd.read_csv("./Data/tickets.csv")
subscriptions_df = pd.read_csv("./Data/subscriptions.csv")

In [ ]:
# Show first 5 rows
tickets_df.head()

In [ ]:
# Show first 5 rows
subscriptions_df.head()

In [ ]:
# Check data type of columns
tickets_df.dtypes

In [ ]:
# Check data type of columns
subscriptions_df.dtypes

### Date format
As you can see the date variables in both dataframes (`datetime_of_purchase`, `start_date`, `end_date`) are still in the string format, so we will have to convert them into the correct **date format** to perform operations on it.

In [ ]:
# Show first 5 rows for tickets_df
tickets_df[['datetime_of_purchase']].head()

In [ ]:
# Show first 5 rows for subscriptions_df
subscriptions_df[['start_date', 'end_date']].head()

Notice that the date variable in tickets_df is in the `YYYY-MM-DD HH:MM:SS` format, while the date variables in subscriptions_df (`start_date`, `end_date`) are in another format (`YYYY-MM-DD`). To get an **overview** of the `strftime()` and `strptime()` **Format Codes** used in `format` parameter go to https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
# Convert the string format to datetime object
tickets_df['datetime_of_purchase'] = pd.to_datetime(tickets_df['datetime_of_purchase'], format='%Y-%m-%d %H:%M:%S')

subscriptions_df['start_date'] = pd.to_datetime(subscriptions_df['start_date'], format='%Y-%m-%d')
subscriptions_df['end_date'] = pd.to_datetime(subscriptions_df['end_date'], format='%Y-%m-%d')

In [ ]:
# Let's check the min and max date in the datasets
print('First date in tickets_df:', tickets_df['datetime_of_purchase'].min())
print('Last date in tickets_df:', tickets_df['datetime_of_purchase'].max())
print('---')
print('First subscription start_date:', subscriptions_df['start_date'].min())
print('Last subscription start_date:', subscriptions_df['start_date'].max())
print('---')
print('First subscription end_date:', subscriptions_df['end_date'].min())
print('Last subscription end_date:', subscriptions_df['end_date'].max())

In [ ]:
# Check how many subscriptions where placed in each month 
monthly_counts = subscriptions_df.groupby(subscriptions_df['start_date'].dt.to_period('M')).size()
monthly_counts

In [ ]:
# Check updated data types
tickets_df.dtypes

In [ ]:
# Check updated data types
subscriptions_df.dtypes

### Missing values
Let's now investigate whether we have to deal with **missing values**:

In [ ]:
# Check number of missing values per variable in tickets_df
for col in tickets_df.columns:
    missings = len(tickets_df[col][tickets_df[col].isnull()])
    print(col, missings)

In [ ]:
# Check number of missing values per variable in tickets_df
for col in tickets_df.columns:
    missings = len(tickets_df[col][tickets_df[col].isnull()]) / float(len(tickets_df))
    print(col, missings)

As you can see we have quite some missing values in the `price`and `customer_id` column. Let's investigate ny filtering for these cases: 

In [ ]:
# Filter for customer_id values and check unique values for ticket_type
print("Filtered: ", tickets_df[tickets_df['customer_id'].isnull()]['ticket_type'].unique())

# Check original set of options for ticket_type
print("Original: ", tickets_df['ticket_type'].unique())

We notice that all the rows for which the `customer_id` is present seem to be **subscriptions** exclusively! 

This makes sense because these are the same `customer_id`s that can be found in the subscriptions dataframe. Therefore, these are the only customers we can have more detailed information about such as (age, contact information, gender, etc.). However, due to strict **GDPR regulations** this information is not shared withing the provided datasets. The other rows in the tickets dataframe can be seen as standalone purchases of tickets (**NOTE**: you also have *Standard Multi* tickets which is a ticket that can be used for a fixed amount (10) of trajectories).

In [ ]:
# Let's also check the missing values for price
print("Filtered: ", tickets_df[tickets_df['price'].isnull()]['ticket_type'].unique())

# Check original set of options for ticket_type
print("Original: ", tickets_df['ticket_type'].unique())

We also notice that in case that a *subscription* was used by the customer, that the `price` of the tickets is **not mentioned** (hence: missing value). The reason for this is that the subscription is paid for upfront at the `start_date`. Both the start date of the subscription as well as the price can be found in the subscription dataframe.

Let's check whether we have to deal with missing values in the **subscription dataframe**:

In [ ]:
# Check number of missing values per variable in subscriptions_df
for col in subscriptions_df.columns:
    missings = len(subscriptions_df[col][subscriptions_df[col].isnull()])
    print(col, missings)

In [ ]:
# Check number of missing values per variable in subscriptions_df
for col in subscriptions_df.columns:
    missings = len(subscriptions_df[col][subscriptions_df[col].isnull()]) / float(len(subscriptions_df))
    print(col, missings)

Nice, it seems like we have don't miss any values in the subscriptions dataset!

### Linking the two datasets
Let's go into more detail about these subscriptions by having a look at them **in both datasets**:

In [ ]:
# Show tickets_df
tickets_df.head()

In [ ]:
# Show subscriptions_df
subscriptions_df.head()

In [ ]:
# Check if all customer_ids are unique in subscriptions_df
subscriptions_df['customer_id'].is_unique

In [ ]:
# Check if number of unique customer_ids in tickets_df is equal to the number of rows (i.e. unique values) in subscriptions_df
len(tickets_df['customer_id'].unique()) == len(subscriptions_df['customer_id'].unique())

# Since we get False, let's have a look at the difference
print("Unique customer IDs in tickets:", len(tickets_df['customer_id'].unique()))
print("Unique customer IDs in subscriptions:", len(subscriptions_df['customer_id'].unique()))

The difference seems to be for just a single value. However, this does not necessarily have to entail an actual Customer ID. Remember from last week that **None variables** are also values! Since we didn't filter for subscriptions, the difference might be due to this value. Let's check:

In [ ]:
# Check if number of unique customer_ids in tickets_df is equal to the number of rows (i.e. unique values) in subscriptions_df
# NOTE: we now filter for non-missing customer ID values!
len(tickets_df[~tickets_df['customer_id'].isnull()]['customer_id'].unique()) == len(subscriptions_df['customer_id'].unique())

Nice, in the end it seems like we do have detailed information about all the subscriptions mentioned in the tickets dataset! You can also check whether the information between the two datasets matches. For example, you can find the class in whihc the customer travels in both datatsets, but is there **ambiguity**?

However, let's investigate some of the logic the datasets a bit further. For example, **how often and when** are the **subscriptions** actually used?

In [ ]:
# Show tickets_df
tickets_df.head()

In [ ]:
# Show tickets_df
subscriptions_df.head()

Let's start by zooming in on a single instance and check whether we can generalize to the entire dataset afterwards. We will focus on customer ID `d232cb95-23cd-45f0-84e7-a53069917f1a` for now:

In [ ]:
# Subset the dataset for one specific customer ID
tickets_df[tickets_df['customer_id'] == 'd232cb95-23cd-45f0-84e7-a53069917f1a']

We notice that this customer (`d232cb95-23cd-45f0-84e7-a53069917f1a`) went from Liège to Ghent every day during the work week, but not during the weekend. Given the hours in the dataset it seems work related, since the timing always fals somewhere in the morning- and evening rush. However, do we find a similar pattern in general for subscription users? Do they **return back** to their start location **on every day**?

In [ ]:
def check_return_trip(day_trips_df, start_col='start_station', end_col='end_station'):
    # Convert to list of (start, end) tuples
    trips = day_trips_df[[start_col, end_col]].to_records(index=False)
    
    # Must have even number of trips
    if len(trips) % 2 != 0:
        return False

    # Check each consecutive pair
    for i in range(len(trips) - 1):
        current_start, current_end = trips[i]
        next_start, next_end = trips[i+1]
        if not (current_start == next_end and current_end == next_start):
            return False

    return True

In [ ]:
# Apply per customer per day
tickets_df['flag'] = None
for cust_id in tickets_df['customer_id'].unique():
    subset_idx = tickets_df[tickets_df['customer_id'] == cust_id].sort_values('datetime_of_purchase').index
    subset_df = tickets_df.loc[subset_idx]
    flag = check_return_trip(subset_df)
    tickets_df.loc[subset_idx, 'flag'] = flag

In [ ]:
# Check for non-NA values
tickets_df[~tickets_df['flag'].isna()]

In [ ]:
# We notice only true values: each customer does indeed return back in the A->B, B->A order
tickets_df[~tickets_df['flag'].isna()]['flag'].unique()

Thus, we can conclude that all customers with a subscription follow the logic of A-->B, B-->A in a chronological order and that we always have complete data without cut-offs!

## 2. Trips & Tickets

In [ ]:
# Let's investigate one of the files in the Trips folder
trips = pd.read_csv("./Data/Trips/Trips_22_09_2025.csv", sep=';')
trips.head()

Each file in the **Trips folder** contains all the trains that were driving on the date captured in the file name. In this example, we are looking at all the trains that departed on **September 22, 2025**. Let's zoom in on a specific using its corresponding `Train number`. We will make use of train with **Train number 3605**. As you can see below, this train was planned to depart in Gent-Sint-Pieters at 05:12:00 AM, but actually departed a few seconds later on 05:12:25 AM. You can also keep track of the delays at each station, the train seems to move between delays (positive values) and being too early (negative values) at different stops. In the end, the train had a delay of just under 4 minutes when arriving in the final station Genk.

In [ ]:
# Filter for train number 3605 and sort based on planned departure time in each train station
trips[trips['Train number']==3605].sort_values('Planned departure time')

In [ ]:
# Count how many stops are on this itinerary
len(trips[trips['Train number']==3605].sort_values('Planned departure time'))

It is important to note that the train does **not** stop in all 39 stations! If you would go to the official website of NMBS (https://www.belgiantrain.be/en/) and look up this specific train, given its departure time and start and end station, you'll notice that the train actually stops in 19 stations. 

<center><img src="./Data/train_3605.png"/></center>
<!-- <img src="train_3605.png" style="display: block; margin: 0 auto"/> -->

So, **where do the remaining 20 stations come from?** The company actually tracks the real-time information of the train at each train station where the train passes, not only the stations where it actually stops. This will help you to visualize the route of the train as well (see later notebooks). For now, including them doesn't help our analysis so we'll drop the stations that are not actual stops. **How?** The stops where the train just passes bybut makes no stop has exact values for `Actual arrival time` and `Actual departure time`as well as for `Planned arrival time` and ` departure time`. We can filter for these conditions as follows:

In [ ]:
# Filter for train number 3605
train_3605 = trips[trips['Train number']==3605].sort_values('Planned departure time')

# Only keep actual stops
train_3605 = train_3605[~(train_3605['Actual arrival time']==train_3605['Actual departure time'])]
train_3605 = train_3605[~(train_3605['Planned arrival time']==train_3605['Planned departure time'])]

# Show
train_3605

This now matches the real journey of the train with **train number 3605**. Also note that for each itinerary the **Planned/Actual departure time of the terminus station** is missing (because it is the last station there is no corresponding departure time). In addition, the opposite also holds: for each itinerary the **Planned/Actual arrival time of the first station** is missing (because it is the first station you can assume that the train is already there). 

Let's now calculate the **average delay at each stations** (in seconds):

In [ ]:
# Average arrival delay in each stop on the itinerary
train_3605['Delay at arrival'].mean()

In [ ]:
# Average departure delay in each stop on the itinerary
train_3605['Delay at departure'].mean()

Let's also do a quick visualization of the **delay evolution of this trajectory**. We will make use of the `matplotlib` package to do some visualizations. Note that we will go into more detail about this package in next week's session.

In [ ]:
# import package
import matplotlib.pyplot as plt

In [ ]:
# Collect arrival delay data from dataframe and convert to list
labels = train_3605['Stopping place'].to_list()
arrival_delays = train_3605['Delay at arrival'].fillna(0).to_list()
x = range(len(labels))

# Width for grouped bars
width = 0.75

fig, ax = plt.subplots(figsize=(12,4))

# Define colors: green if negative, red if positive
colors = ['green' if v < 0 else 'red' for v in arrival_delays]

# Plot bars and horizontal line at 0
plt.bar(x, arrival_delays, width, color=colors)
plt.axhline(0, color="black", linewidth=0.8) 

# Set title and x labels
plt.title("Evolution of arrival delay at each stop for train number 3605")
plt.xticks(x, labels, rotation=90)
plt.show()

In [ ]:
# Collect departure delay data from dataframe and convert to list
labels = train_3605['Stopping place'].to_list()
arrival_delays = train_3605['Delay at departure'].fillna(0).to_list()
x = range(len(labels))

# Width for grouped bars
width = 0.75

fig, ax = plt.subplots(figsize=(12,4))

# Define colors: green if negative, red if positive
colors = ['green' if v < 0 else 'red' for v in arrival_delays]

# Plot bars and horizontal line at 0
plt.bar(x, arrival_delays, width, color=colors)
plt.axhline(0, color="black", linewidth=0.8) 

# Set title and x labels
plt.title("Evolution of departure delay at each stop for train number 3605")
plt.xticks(x, labels, rotation=90)
plt.show()

## 3. Stations & Stops

In [ ]:
# Import stations dataset
stations_df = pd.read_csv("./Data/stations.csv")
stations_df.head()

In [ ]:
# Import stops dataset
stops_df = pd.read_csv("./Data/stops.csv").drop_duplicates('URI')
stops_df.head()

In the stations dataset you can find the `station_id` identifier. This is a **unique code** for each train station and is used to link different datasets together as shown in the ERD at the beginning of this notebook. However, this is not yet readily available in the stops dataset. It can be retreived from the `URI` (Uniform Resource Identifier) or `parent_stop` column. As you can see a station can have multiple platforms, let's investigate this in more detail:

In [ ]:
def extract_station_id_from_parent_stop(text):
    return text.split('/')[-1]

# Apply function on each row
stops_df['station_id'] = stops_df['parent_stop'].apply(lambda x: extract_station_id_from_parent_stop(x)) 

In [ ]:
def extract_station_id_from_URI(text):
    return text.split('/')[-1].split('#')[0]

# Apply function on each row
stops_df['station_id2'] = stops_df['URI'].apply(lambda x: extract_station_id_from_URI(x)) 

In [ ]:
# Show result
stops_df[['station_id', 'station_id2', 'URI', 'parent_stop', ]].head()

We can now use this unique station identifier (`station_id`) to get station specific information. For example, we can check **how many platforms** are provided at each train station:

In [ ]:
# Count the number platforms for each station
platform_count_df = stops_df.groupby('station_id').size().reset_index()

# Rename columns
platform_count_df.columns = ['station_id', 'platforms']

# Order based on count
platform_count_df = platform_count_df.sort_values('platforms', ascending=False)

# Show
platform_count_df

In [ ]:
# Convert station_id from string type to integer type
platform_count_df['station_id'] = platform_count_df['station_id'].astype(int)

In [ ]:
# Merge result with stations_df's station_id to retrieve the names of the station
platform_count_df = platform_count_df.merge(stations_df[['station_id', 'name']], how='left', on='station_id')
platform_count_df

In [ ]:
# Check for missing values
platform_count_df.isnull().sum()

Nice! We have the number of platforms for each train station in our dataset. Thank to the inclusion of the station name we can also **verify** this information via external data (e.g., wikipedia).

## 4. Travelers

In [ ]:
# Import travelers dataset
travelers_df = pd.read_csv("./Data/travelers.csv", sep=";", index_col=0)
travelers_df.head()

In [ ]:
# Let's rename for convenience later
travelers_df = travelers_df.rename({"Avg number of travelers in the week": "week",
                                    "Avg number of travelers on Saturday": "saturday",
                                    "Avg number of travelers on Sunday": "sunday"}, axis=1)

In [ ]:
# Check number of missing values per variable
for col in travelers_df.columns:
    missings = len(travelers_df[col][travelers_df[col].isnull()]) / float(len(travelers_df))
    print(col, missings)

In [ ]:
# Change settings to visualize ALL rows
pd.set_option('display.max_rows', None)
print(travelers_df[travelers_df.isnull().any(axis=1)])

# Change settings back
pd.reset_option('display.max_rows')

Interestingly, we never witness a row where all values are completely missing. Further inspection on Wikipedia and the NMBS website reveal that there are no train rides on these dates for these stations. For example, *Baasrode-Zuid* & *Buda* only have train rides during the week and none in the weekend. Therefore, we will **impute every missing value with zero**.

In [ ]:
# Impute missing values with zero
travelers_df['week'].fillna(0)
travelers_df['saturday'].fillna(0)
travelers_df['sunday'].fillna(0)

# Show
travelers_df.head()

In [ ]:
# Create total travelers over the week
travelers_df["week_total"] = 5 * travelers_df["week"] + travelers_df["saturday"] + travelers_df["sunday"]

In [ ]:
# Get weekend avg
travelers_df["weekend"] = (travelers_df["sunday"] + travelers_df["saturday"]) / float(2)

In [ ]:
# Get avg travelers per day (including weekends)
travelers_df["avg_day"] = travelers_df["week_total"] / float(7)

In [ ]:
# Check top 10 stations with highest number of travelers during the weekend
travelers_df.sort_values(by="week", ascending=False)[["Station", "week"]].head(10)

In [ ]:
# Check top 10 stations with highest number of travelers during the week
travelers_df.sort_values(by="weekend", ascending=False)[["Station", "weekend"]].head(10)

Most remarkable differences are between Brussels Midi and Brussels North. **Brussels North** is in the middle of the business center, attracting many commuters during the week, while **Brussels Midi** is the most important international railway station of Belgium and thus attracts many tourists (mainly during the weekend). 

Also notice how **Antwerpen** and **Leuven** almost have equal travelers during the week and are **off by almost a factor of two** during the weekend. This could signify a more or less equal commute potential but a far greater touristic potential for Antwerp. However, both are educated guesses based on my personal knowledge about the country. Implementing this mathematically and on a larger scale requires **external data**!

Other explanations for the commute numbers may possible lay in the number of facilities provided by each station. As a proof of concept, we'll try to mathematically proof whether weekly commute numbers are linked to availabilty of free parking and/or tram stations **next week**.

## 5. Facilities

In [ ]:
# Import facilities
facilities = pd.read_csv("./Data/facilities.csv")

# Show
facilities.head()

In [ ]:
# Check number of missing values per variable
for col in facilities.columns:
    missings = len(facilities[col][facilities[col].isnull()]) / float(len(facilities))
    print(col, missings)

In [ ]:
# Check data type of columns
facilities.dtypes

In [ ]:
# Let's have a look at the sales_open_monday column
facilities[['sales_open_monday']]

In [ ]:
# Change column to time object
facilities['sales_open_monday2'] = pd.to_datetime(facilities['sales_open_monday'].astype(str), format='%H:%M') - pd.to_datetime('00:00', format='%H:%M')

In [ ]:
# For instance only use subset of 'late openers' (i.e. opened later than 8 AM)
late_openers = facilities[facilities['sales_open_monday2'] > pd.Timedelta(8,'h')]

In [ ]:
# Show late openers on Monday
late_openers[['name', 'sales_open_monday']]

## Group Assignment

Although we haven't covered all the datasets of the assignment in this notebook, it already helps you to get started with the group assignment. Moreover, based on this week's course material you are already able to solve some of required subquestions found on Ufora.